In [ ]:
%matplotlib notebook 
from model.resattnet import SelfPose
from torch.utils.data import DataLoader
import time
import copy
import torch
import numpy as np
import matplotlib.pyplot as plt
import glob
from mpl_toolkits.mplot3d import axes3d 
import typing as tp
import numpy as np
import matplotlib.pyplot as plt
from dataloader_real import DomeMaster


In [ ]:
test_loader = DomeMaster("../Datasets/domemaster/theta_chest_domemaster/")
test_loader = DataLoader(test_loader, num_workers=1, batch_size=1)
self_pose = SelfPose()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") 
# self_pose.load_state_dict(torch.load("./ckpts/vae_no_kld/10_epoch_1.6332732890902123e-05.pth"))
self_pose.to(device)
print()

In [ ]:

def get_chain_dots(
        joints: np.ndarray,   # shape == (n_dots, 3)
        chain_dots_indexes: tp.List[int], # length == n_dots_in_chain
                                          # in continuous order, i.e. 
                                          # left_hand_ix >>> chest_ix >>> right_hand_ix
        ) -> np.ndarray:    # chain of dots
    print(joints.shape)
    return joints[chain_dots_indexes]


def get_chains(
        joints,   # shape == (n_dots, 3)
        head: tp.List[int], # pelvis >>> chest >>> head
        spine: tp.List[int], # left_leg >>> pelvis >>> right_leg
        arm1: tp.List[int],
        arm2,
        neck,
        right_leg,
        left_leg,
        ):
    return (get_chain_dots(joints, head),
            get_chain_dots(joints, spine),
            get_chain_dots(joints, arm1),
            get_chain_dots(joints, arm2),
            get_chain_dots(joints, neck),
            get_chain_dots(joints, right_leg),
            get_chain_dots(joints, left_leg),
            )


def subplot_nodes(dots: np.ndarray, # shape == (n_dots, 3)
                  ax):
    return ax.scatter3D(*dots.T, c=dots[:, -1], s=20)


def subplot_bones(chains: tp.Tuple[np.ndarray, ...], ax):
    return [ax.plot(*chain.T) for chain in chains]


def plot_skeletons(skeletons, chains_ixs):
    joints = skeletons[0]
    fig = plt.figure(figsize=(10, 5))
    chains = get_chains(joints, *chains_ixs)
    ax = fig.add_subplot(1, 1, 1, projection='3d')
    subplot_nodes(joints, ax)
    subplot_bones(chains, ax)
    plt.show()
    
def plot(pred_pose):
    # Kinematic Tree for visualization
    # each list create bones between joints
    pred = np.array(
    [[  0,           0,           0        ],
     [  4.22492806, -22.67886381,  50.55490345],
     [  2.8284324,  -22.74109124,  38.9576961 ],
     [  1.14356441, -22.34996218,  25.72032099],
     [-14.35919458, -16.56799695,  20.91153744],
     [-15.43520405, -24.19927021,  40.86328072],
     [-15.73011997, -26.19144252,  65.89507353],
     [ 15.4553649,  -24.21402085,  17.68834905],
     [ 18.53139455, -29.87654767,  38.08859206],
     [ 25.34144849, -29.78593198,  62.24731525],
     [ -0.751921,   -21.90995527,  10.82824809],
     [ -1.87156293, -20.19901538,  66.8921517 ],
     [  0.60275076, -18.47801771, 106.73545244],
     [  3.14211695, -32.39483848, 139.29781386],
     [ 13.83222249, -24.57550126,  65.14085695],
     [ 19.37489383, -25.83197523, 104.73484489],
     [ 18.21862104, -36.73699566, 138.4111168 ],])
    pred[4:10] = pred_pose
    chains_ixs = ([0],[1, 2, 3],[3, 4, 5, 6],[3, 7, 8, 9],[3, 10],[1, 11,12,13],[1, 14,15,16]) 
    plot_skeletons([pred], chains_ixs)
 

In [ ]:
for i, (img, _, _) in enumerate(test_loader):
    if i != 21:
        continue
    
    img1 = np.transpose(img.detach().numpy().reshape(3,368, 368), [1,2,0])
    plt.imshow(img1)
    plt.show()
    img = img.cuda()
    output, _, _, _, _ = self_pose(img)
    output = (output[-1].reshape(6,3) * 300).cpu().detach().numpy()
    plot(output)
    print(output)
    0/0

In [ ]:
from pytorchcv.model_provider import get_model as p
model = p("resattnet56", pretrained=False)
import torch.nn as nn
state = torch.hub.load_state_dict_from_url("https://github.com/phamquiluan/ResidualAttentionNetwork/releases/download/v0.1.0/resattnet56.pth")
# mo

model.load_state_dict(state["state_dict"])

model

In [ ]:
model1 = nn.Sequential(*list(model.features.children())[:-1])

In [ ]:
for name, param in model.named_parameters():
    print(name, param)
    break
    

In [ ]:
state = torch.hub.load_state_dict_from_url("https://github.com/phamquiluan/ResidualAttentionNetwork/releases/download/v0.1.0/resattnet56.pth")
# mo

model1.load_state_dict(state["state_dict"])


In [ ]:
list(model1.named_children())

In [ ]:
list(model.named_children())